In [1]:
import os
import sys

import numpy as np
#from skimage import io
#import skimage
import matplotlib.pyplot as plt
#from meta_load import *

sys.path.append(os.path.join("..",'Data_prep'))
sys.path.append(os.path.join("..",'Models2'))

from Custom_dataloader import *
import pandas as pd
import matplotlib.pyplot as plt
import pickle

DB="/run/user/1000/gvfs/afp-volume:host=MyCloudPR4100.local,user=aorus_2,volume=Paltas_DataBase"
Dbd="/run/user/1000/gvfs/afp-volume:host=MyCloudPR4100.local,user=aorus_2,volume=Paltas_DataBase/Data_Base"
#DBd="//MYCLOUDPR4100/Paltas_DataBase/Data_Base"
#DB="//MYCLOUDPR4100/Paltas_DataBase"
#d_t=transforms.Compose([phantom_segmentation(False)])

datar=Dataset_direct(root_dir=Dbd,ImType=['PhantomRGB'],Intersec=False,retrieve_img=False)

pd.options.display.max_columns = None

In [21]:
os.listdir(os.path.join(DB,"metadata"))
meta=['bl_per_wmask_phantom','stat_val_hsv_wmask_phantom','stat_val_lab_wmask_phantom']
ddlist=np.array(os.listdir(os.path.join(DB,"metadata",meta[0])))
dlist=np.vectorize(lambda d:d.split(".")[0])(ddlist)

In [ ]:
date=np.vectorize(lambda d:("_").join(datar[np.where(datar.aID==d)[0][0]]["Date"].split("_")[:2]))(dlist)
Y=np.vectorize(lambda d:datar[np.where(datar.aID==d)[0][0]]["landmarks"])(dlist)
timei=np.vectorize(lambda d:datar[np.where(datar.aID==d)[0][0]]['PhantomRGB_metadata']['DateTime'])(dlist)

In [ ]:
dat=np.vectorize(lambda d:(pd.read_pickle(os.path.join(DB,"metadata",meta[0],d)))[d.split('.')[0]])(ddlist).reshape(-1,1)
camp=np.array(['mean','std','mode'])
dat1=np.vectorize(lambda d,c:(pd.read_pickle(os.path.join(DB,"metadata",meta[1],d)))[d.split('.')[0]][c],
            signature="(),()->()")(ddlist.reshape(-1,1),camp)
dat2=np.vectorize(lambda d,c:(pd.read_pickle(os.path.join(DB,"metadata",meta[2],d)))[d.split('.')[0]][c],
            signature="(),()->()")(ddlist.reshape(-1,1),camp)


In [ ]:
date=np.vectorize(lambda d:("_").join(datar[np.where(datar.aID==d)[0][0]]["Date"].split("_")[:2]))(dlist)

# VAE GEN

In [2]:
meta_dir="/run/user/1000/gvfs/afp-volume:host=MyCloudPR4100.local,user=aorus_2,volume=Paltas_DataBase/metadata_VAE_v2"
metad=np.array(os.listdir(meta_dir)[1:])
metad_dict=np.vectorize(lambda meta: pickle.load(open(os.path.join(meta_dir,meta),'rb')))(metad)
meta_u=np.vectorize(lambda dic:dic['u'],signature='()->(j)')(metad_dict)
meta_sig=np.vectorize(lambda dic:dic['sig'],signature='()->(j)')(metad_dict)
meta_C=np.vectorize(lambda dic:dic['Class'][0],otypes=[object],signature='()->()')(metad_dict)
meta_D=np.vectorize(lambda dic:dic['Date'][0],otypes=[object],signature='()->()')(metad_dict)

In [6]:
metad_dict

array([{'u': array([[0.07869902, 0.        , 0.14296825, 0.2914372 , 0.19480088,
        0.2523863 , 0.28069866, 0.19477297, 0.        , 0.        ,
        0.17233148, 0.        , 0.04948647, 0.02155615, 0.7284131 ,
        0.13793877, 0.34823874, 0.35693416, 0.        , 0.13558303]],
      dtype=float32), 'sig': array([[0.        , 0.3025026 , 0.        , 0.4355199 , 0.16388698,
        0.4148169 , 0.        , 0.4654258 , 0.44232717, 0.52275866,
        0.59687054, 0.17484462, 0.00303755, 0.        , 0.5869827 ,
        0.        , 0.35440218, 0.14268796, 0.5155141 , 0.64862484]],
      dtype=float32), 'Place': ['10_A'], 'Date': ['11_junio_1'], 'Class': ['N_Control']},
       {'u': array([[0.60876954, 0.        , 0.        , 1.3173093 , 0.8593364 ,
        0.47003186, 0.40660205, 0.93393236, 0.5505697 , 0.        ,
        1.1062106 , 0.16151091, 0.        , 0.36891168, 0.09802139,
        0.        , 0.31025362, 0.82567316, 0.        , 0.91215366]],
      dtype=float32), 'sig': arra

# REG

In [ ]:
datafd={
    'ID':dlist,
    'lab_mean':dat2[:,0],
    'lab_std':dat2[:,1],
    'lab_mode':dat2[:,2],
    'hsv_mean':dat1[:,0],
    'hsv_std':dat1[:,1],
    'hsv_mode':dat1[:,2],
    'black_per':dat.reshape(-1,),
    'hour':np.vectorize(lambda cdt:cdt.split(' ')[1].split(':')[0])(timei),
    'date':date,
    'Y':Y
}
data=pd.DataFrame(datafd)
data

In [ ]:
data.to_csv('/home/liarpi2/Escritorio/metadata_VAE_v2.csv')

In [ ]:
print(datar[0]['PhantomRGB_metadata']['DateTime'])
print(datar[5]['PhantomRGB_metadata']['DateTime'])

In [ ]:
#Normalize
import sklearn
from sklearn.preprocessing import StandardScaler
cont=['lab_mean','lab_std','lab_mode','hsv_mean','hsv_std','hsv_mode','black_per']
features = data[cont]


scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
data[cont]=features
data

In [ ]:
#Encode time and day
data=pd.concat([data,pd.get_dummies(data['hour'],columns='hour'),pd.get_dummies(data['date'],columns='date')],axis=1)
data

In [ ]:
hour_cl=np.unique(np.array(data['hour']))
print(hour_cl)
day_cl=np.unique(np.array(data['date']))
print(day_cl)
data.columns

In [ ]:
#Clasificación para detección de deficiencia hidrica
data=data[(data['Y']=='Control')|\
          #(data['Y']=='K_Control')|\
          #(data['Y']=='N_Control')|\
          #(data['Y']=='P_Control')|\
          (data['Y']=='H50%')\
           #(data['Y']=='H75%')\
         ]
#X=data[list(data.columns[1:7]) + list(data.columns[16:])]
X=data[list(data.columns[1:7])]
print(X.columns)
Yo=((data['Y']=='Control') | (data['Y']=='K_Control') | (data['Y']=='N_Control') | (data['Y']=='P_Control')).astype(int)
Yo


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

GR=np.arange(1,6)
def LR_GR(GR,X,Y):
    polynomial_features = PolynomialFeatures(degree=GR)
    r=polynomial_features.fit_transform(X[:,:6])
    Xco=np.hstack((r,np.array(X[:,6:])))
    X_train, X_test, y_train, y_test = train_test_split(Xco, Y, random_state=20,train_size=0.8)
    Lr1=LogisticRegression(max_iter=4000,C=5)
    Lr1.fit(X_train,y_train)
    pr_tr=Lr1.score(X_train,y_train)
    pr_ts=Lr1.score(X_test,y_test)
    print(GR)
    print(' R^2_train: ', pr_tr)
    print(' R^2_test: ', pr_ts)
    return Lr1,GR,pr_tr,pr_ts
v_LR_GR=np.vectorize(LR_GR,signature='(),(j,k),(l)->(),(),(),()')
LR,GR,pr_tr2,pr_ts2=v_LR_GR(GR,X,np.array(Yo))

plt.plot(GR,pr_tr2, 'ro--')
plt.plot(GR,pr_ts2, 'bo--')
plt.show()

plt.bar(np.arange(LR[0].coef_[0].shape[0]),np.abs(LR[0].coef_[0]))
plt.show()

plt.bar(np.arange(LR[0].coef_[0].shape[0]),np.abs(LR[0].coef_[0].sort()))
plt.show()

In [ ]:
polynomial_features = PolynomialFeatures(degree=1)
Xc=polynomial_features.fit_transform(X[X.columns[:6]])
Xc=np.hstack((X[X.columns[6:]],Xc))
C=np.arange(0.001,3,0.1)
def LR_C(C,X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=20,train_size=0.8)
    Lr1=LogisticRegression(max_iter=4000,C=C)
    Lr1.fit(X_train,y_train)
    pr_tr=Lr1.score(X_train,y_train)
    pr_ts=Lr1.score(X_test,y_test)
    print(C)
    print(' R^2_train: ', pr_tr)
    print(' R^2_test: ', pr_ts)
    return Lr1,C,pr_tr,pr_ts
v_LR_C=np.vectorize(LR_C,signature='(),(j,k),(l)->(),(),(),()')

LR,C,pr_tr1,pr_ts1=v_LR_C(C,Xc,np.array(Yo))

plt.plot(C,pr_tr1, 'ro--')
plt.plot(C,pr_ts1, 'bo--')
plt.show()

In [ ]:
C[np.argmax(pr_ts1)]

In [ ]:
LR[0]

In [ ]:
#np.hstack((np.array(polynomial_features.get_feature_names(X.columns[:6])),np.array(X.columns[6:])))[np.abs(LR[0].coef_[0])>0.2]
np.hstack((np.array(polynomial_features.get_feature_names(X.columns[:6]))))[np.abs(LR[0].coef_[0])>0.1]

In [15]:
from sklearn.tree import DecisionTreeClassifier

DP=np.arange(4,30)
mSL=np.arange(1,70,2)
MSL=np.arange(2,25,1)

#VARIANDO DP
def DT_DP(DP,X,Y):
  X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=20,train_size=0.8)
  Lr1 = DecisionTreeClassifier(max_depth=DP,min_samples_leaf=21,max_leaf_nodes=6)#-----------------------------------------
  #Lr1 = DecisionTreeClassifier(max_depth=DP,min_samples_leaf=8)
  Lr1.fit(X_train,y_train)
  pr_tr=Lr1.score(X_train,y_train)
  pr_ts=Lr1.score(X_test,y_test)
  print(DP)
  print(' R^2_train: ', pr_tr)
  print(' R^2_test: ', pr_ts)
  return Lr1,DP,pr_tr,pr_ts
v_DT_DP=np.vectorize(DT_DP,signature='(),(j,k),(l)->(),(),(),()')

DT1,DP,pr_tr1,pr_ts1=v_DT_DP(DP,X,np.array(Yo))

#VARIANDO mSL
def DT_mSL(mSL,X,Y):
  X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=20,train_size=0.8)
  Lr1 = DecisionTreeClassifier(max_depth=5,min_samples_leaf=mSL,max_leaf_nodes=6)#-----------------------------------------
  #Lr1 = DecisionTreeClassifier(min_samples_leaf=mSL)
  Lr1.fit(X_train,y_train)
  pr_tr=Lr1.score(X_train,y_train)
  pr_ts=Lr1.score(X_test,y_test)
  print(mSL)
  print(' R^2_train: ', pr_tr)
  print(' R^2_test: ', pr_ts)
  return Lr1,mSL,pr_tr,pr_ts
v_DT_mSL=np.vectorize(DT_mSL,signature='(),(j,k),(l)->(),(),(),()')

DT2,mSL,pr_tr2,pr_ts2=v_DT_mSL(mSL,X,np.array(Yo))

#VARIANDO MSL
def DT_MSL(MSL,X,Y):
  X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=20,train_size=0.8)
  Lr1 = DecisionTreeClassifier(max_depth=5,min_samples_leaf=21,max_leaf_nodes=MSL)#-----------------------------------------
  #Lr1 = DecisionTreeClassifier(max_leaf_nodes=MSL,min_samples_leaf=8)
  Lr1.fit(X_train,y_train)
  pr_tr=Lr1.score(X_train,y_train)
  pr_ts=Lr1.score(X_test,y_test)
  print(MSL)
  print(' R^2_train: ', pr_tr)
  print(' R^2_test: ', pr_ts)
  return Lr1,MSL,pr_tr,pr_ts
v_DT_MSL=np.vectorize(DT_MSL,signature='(),(j,k),(l)->(),(),(),()')

DT3,MSL,pr_tr3,pr_ts3=v_DT_MSL(MSL,X,np.array(Yo))



plt.plot(DP,pr_tr1, 'ro--')
plt.plot(DP,pr_ts1, 'bo--')
plt.show()

plt.plot(mSL,pr_tr2, 'ro--')
plt.plot(mSL,pr_ts2, 'bo--')
plt.show()

plt.plot(MSL,pr_tr3, 'ro--')
plt.plot(MSL,pr_ts3, 'bo--')
plt.show()

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
DT3[5]

In [16]:
sklearn.tree.plot_tree(DT3[6],feature_names=X.columns)

NameError: name 'sklearn' is not defined

In [ ]:
df=load_meta_v2()
df